## Set up

In [65]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from gensim.models import Phrases
from gensim.models.phrases import Phraser 
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle

In [32]:
data = pd.read_csv("data/cleaned_data1.csv")

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_6447/1751319474.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/cleaned_data1.csv")


## Preprocessing

Tokenizer

In [33]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
data["tokenized"] = data["cleaned_text"].apply(lambda x: tokenizer.tokenize(x))

Lemmatizer & POS filtering

In [34]:
#https://www.h2kinfosys.com/blog/part-of-speech-tagging-chunking-with-nltk/
def get_wordnet_pos(tag):
    """Convert POS tag to a format recognized by WordNetLemmatizer"""
    if tag.startswith('J'): #NLTK tags adjectives as JJ (normal), 
                            #JJR (comparative) and JJS (superlative)
        return wordnet.ADJ  #and then it can be mapped to the wordnet database
                              #to get the lemma
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Lemmatizes tokens with POS tagging"""
    tagged_tokens = pos_tag(tokens)  
    return [lemmatizer.lemmatize(word, pos) for word, tag in tagged_tokens
            if (pos := get_wordnet_pos(tag))]

data["lemmatized"] = data["tokenized"].apply(lemmatize_tokens)

Stop words

In [54]:
#Custom defined list
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "wasnt", "yeah", "okay", "lol",
        "gt", "cuz", 'id']

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "op", "fuck", "shit"]

topic_specific = ["self", "improvement", "selfimprovement", "rselfimprovement", 
    'even', 'time', 'bad', 'best', 'never', 'well', 'hard', 'always', 'help', 'first',
    'back', 'problem', 'right', 'try', 'need', 'actually', 'everything', 'long', 'sure', 
    'care', 'look', 'everyone', 'enough', 'else', 'great', 'point', 'kind', 
    'advice', 'now', 'year', 'old', 'month', 'young', 'age', 'last', 'later', 
    'ago', 'late', 'future', 'finally', 'early', 'bc', 'decade', 
    'tbh', 'spent', 'almost', 'ur', 'others', 'experience', 'change', 'world', 'different', 
    'important', 'positive', 'negative', 'value', 'question', 'moment', 'often', 'way', 
    'instead', 'situation', 'rather', 'understand', 'personal', 'practice', 'start', 
    'week', 'small', 'hour', 'use', 'easy', 'minute', 'break', 'daily', 'night', 'today', 
    'everyday', 'list', 'morning', 'task', 'spend',
    'everybody', 'everyone', 'someone', 'somebody', 'anything', 'everything', 'nothing', 'something',
    'good', 'great', 'fine', 'better', 'best', 'okay', 'alright', 'just', 'only', 'simply', 'actually', 
    'careful', 'attentive', 'caring', 'concerned', 
    'positive', 'negative', 
    'important', 'necessary', 'essential', 'key', 'significant', 'irrelevant', 
    'unimportant', 'worthwhile', 'change', 'alter', 'modify', 'adjust', 'shift', 'transform', 
    'problem', 'issue', 'challenge', 'difficulty', 'obstacle', 'struggle', 'complication', 'solution', 
    'answer', 'remedy', 'plan', 'idea', 'approach', 'method', 'strategy', 'option', 'choice', 'decision', 
    'situation', 'circumstance', 'condition', 'event', 'experience', 'case', 'example', 'scenario', 
     'case', 'step', 'action', 'move', 'process', 'procedure', 'approach', 'method', 'result', 'outcome', 'consequence', 
    'conclusion', 'end', 'finish', 'start', 'begin', 'initiate', 'open', 'launch', 'beginning', 'commencement',
    'day', 'one', 'happy', 'little', 'big', 'probably', 'reason', 'able', 'away', 'sometimes', 
    'ever', 'matter', 'real', 'bit', 'already', 'anyone', 'especially', 'definitely', 
    'normal', 'whole', 'comment', 'honestly', 'completely', 'literally', 'sorry', 'eventually', 
    'day', 'part', 
    'one', 'happy', 'little', 'big', 'probably', 'reason', 'able', 'away', 'sometimes', 
    'ever', 'matter', 'real', 'bit', 'already', 'anyone', 'especially', 'definitely', 
    'normal', 'whole', 'comment', 'honestly', 'completely', 'literally', 'sorry', 'eventually', 
    'day', 'part', 'wrong', 'pretty', 'luck', 'true', 'fact', 'far', 'sense', 'possible', 
    'however', 'word', 'mean', 'likely', 'opinion', 'story', 'side', 'gonna', 'certain', 
    'personally', 'chance', 'second', 'absolutely', 'forward', "thanks", "place", "less",
     "least"]

stop_words = english + reddit + topic_specific

In [55]:
#Remove stopwords
def remove_stopwords(tokens):
    """Removes stopwords"""
    return [token for token in tokens if token not in stop_words]

# Apply function to column
data["filtered"] = data["lemmatized"].apply(remove_stopwords)

Bigrams

In [56]:
# Train bigram model on the lemmatized column
bigram = Phrases(data["filtered"], min_count=20)
bigram_phraser = Phraser(bigram)  # Optimizes since it is a lighter-weight
                                   #version of Phrases

def add_bigrams(tokens):
    """Adds bigrams to a list of tokens if they appear frequently enough."""
    bigram_tokens = bigram_phraser[tokens]
    return tokens + [token for token in bigram_tokens if '_' in token]

data["ngrams"] = data["filtered"].apply(add_bigrams)

Filtering of too rare of too common words

In [57]:
# Convert df column to a list of lists for later steps
docs = data["ngrams"].tolist() 

# Create a dictionary from the tokenized documents
dictionary = Dictionary(docs)

# Filter out words that appear in fewer than 20 documents or 
#  more than 80% of documents, and keep the 1500 most frequent
dictionary.filter_extremes(no_below=20, no_above=0.8, keep_n=1500)


Vectorize

In [58]:
# Create bag of words representation
corpus = [dictionary.doc2bow(doc) for doc in docs]

Save to disk

In [66]:
# Save corpus and dictionary together
with open('corpus_and_dict_v2.pkl', 'wb') as f:
    pickle.dump((corpus, dictionary), f)


## Training

### Test ?

In [62]:
#Set hyperparameters
num_topics = 10
chunksize = 2500 
passes = 25
iterations = 75 
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  
model = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model.top_topics(corpus)  
pprint(top_topics)
cm = CoherenceModel(model=model, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) 
print(cm.get_coherence())  
model.save("lda_models/model35") #CHANGE 

[([(0.106128566, 'friend'),
   (0.06418519, 'relationship'),
   (0.05232138, 'new'),
   (0.024933714, 'alone'),
   (0.024494888, 'hobby'),
   (0.024372458, 'partner'),
   (0.020313695, 'interest'),
   (0.020275252, 'group'),
   (0.018838152, 'fun'),
   (0.01639664, 'conversation'),
   (0.01638945, 'talk'),
   (0.01630708, 'nice'),
   (0.015207275, 'family'),
   (0.015056772, 'together'),
   (0.0127254315, 'close'),
   (0.012379697, 'comfortable'),
   (0.011681595, 'interested'),
   (0.011521859, 'connection'),
   (0.011101762, 'anymore'),
   (0.010666552, 'couple')],
  -2.7198844342114423),
 ([(0.014256217, 'mind'),
   (0.01363133, 'thought'),
   (0.013284656, 'feeling'),
   (0.008065681, 'love'),
   (0.007662425, 'human'),
   (0.007264539, 'brain'),
   (0.0061489264, 'emotion'),
   (0.006119294, 'strong'),
   (0.0060303905, 'head'),
   (0.0058391686, 'effort'),
   (0.005631324, 'worth'),
   (0.0055919816, 'journey'),
   (0.0051781987, 'control'),
   (0.0048400583, 'mindset'),
   (0.00

### LDAVis

In [63]:
top_topics = model.top_topics(corpus) 
pyLDAvis.enable_notebook()
# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_viz)



### Check topic probability distribution

In [64]:
data["topic_distribution"] = [model.get_document_topics(bow) for bow in corpus]
data["dominant_topic"] = [max(model.get_document_topics(bow), key=lambda x: x[1])[0] for bow in corpus]
data["prob_topic"] = [max(model.get_document_topics(bow), key=lambda x: x[1])[1] for bow in corpus]

In [68]:
data.dominant_topic.value_counts()

dominant_topic
0    419442
3     24732
1     19896
2     11019
4      9513
6      9250
5      5540
7      4245
9      1549
8      1388
Name: count, dtype: int64

In [71]:
num_topics = model.num_topics
for topic_id in range(num_topics):
    topic_words = model.show_topic(topic_id, topn=15)
    top_words_str = ", ".join(word for word, prob in topic_words)
    print(f"Topic {topic_id}: {top_words_str}")

Topic 0: mind, thought, feeling, love, human, brain, emotion, strong, head, effort, worth, journey, control, mindset, attention
Topic 1: goal, habit, gym, sleep, activity, routine, exercise, motivation, meditation, progress, phone, bed, walk, discipline, room
Topic 2: job, school, high, skill, college, career, business, community, dream, study, degree, position, company, passion, art
Topic 3: friend, relationship, new, alone, hobby, partner, interest, group, fun, conversation, talk, nice, family, together, close
Topic 4: body, healthy, health, mental, weight, porn, food, physical, mental_health, addiction, water, diet, drug, muscle, fat
Topic 5: social, game, medium, video, social_medium, class, youtube, music, sport, movie, club, video_game, apps, high_school, phone
Topic 6: woman, girl, man, men, guy, sex, confidence, attractive, date, ugly, confident, personality, face, dude, sexual
Topic 7: money, parent, kid, home, family, child, house, mom, car, adult, country, wife, brother, dad

## Exploration

In [ ]:
#Function if I want to filter
#topicx = data[(data.dominant_topic == x) & (data.prob_topic > 0.20)].sort_values("prob_topic", ascending=False)

In [78]:
def show_example_docs(df, topic_n, ascending):
    df = data[(data.dominant_topic == topic_n)].sort_values("prob_topic", ascending=ascending)
    print(df["type"].value_counts())
    with pd.option_context('display.max_colwidth', None):
        display(df[["id","type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])
    return None

## Topic 0
Topic 0: mind, thought, feeling, love, human, brain, emotion, strong, head, effort, worth, journey, control, mindset, attention

Docs with highest probability: GOOD

Docs with lowest probability: Not too bad

In [97]:
topic0 = data[(data.dominant_topic == 0)].sort_values("prob_topic", ascending=False)
print(topic0["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic0[["id","type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       342078
submission     77364
Name: count, dtype: int64


id        type  \
15259    ackypy  submission   
32812    ipcwh1  submission   
99479   cd4dtte     comment   
337092  ibfm1hv     comment   
67661    zcrphj  submission   
207433  fl9b0k8     comment   
441190  jo5gart     comment   
211202  fon3h97     comment   
34435    jmryxs  submission   
214014  fr65e2u     comment   
35403    k49y9h  submission   
25653    ewyu1b  submission   
57838    utjib1  submission   
11023    8mgbjx  submission   
363310  irqr72i     comment   
32839    iprrnu  submission   
281736  hbcl8ay     comment   
191869  f4zx2t6     comment   
219194  fws6lmj     comment   
32798    iozjit  submission   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

## Topic 1
Topic 1: goal, habit, gym, sleep, activity, routine, exercise, motivation, meditation, progress, phone, bed, walk, discipline, room

Docs with highest probability: GOOD

Docs with lowest probability: Good enough!

In [98]:
topic1 = data[(data.dominant_topic == 1)].sort_values("prob_topic", ascending=False)
print(topic1["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic1[["id","type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       15667
submission     4229
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
70853,10h1kgh,submission,obviously i wont be able to follow it to a t every time but i want this to be the basis of my improvement monday wake up am min walk am min shower am min fresh up am min breakfast am leave for work by am min walk pm work until pm min hiit exercise move ya body pm min dinner pm free time pm bedtime routine tuesday wake up am min walk am min shower am min fresh up am min breakfast am leave for work by am min walk pm work until pm gym for min min dinner free time pm bedtime routine pm wednesday wake up am min walk am min shower am min fresh up am min breakfast am leave for work by am work until pm write for min min dinner pm free time pm bedtime routine pm thursday repeat monday friday repeat tuesday saturday free day sunday reset day clean living space do outside activity,1,0.818906
183781,evdq1bs,comment,maybe your first goal should be to take care of your sleep schedule this was one of my first undertakings as part of my self improvement journey and it helped me massively it was a keystone habit a habit that allowed me to build more habits off of it once i got it down pat build good sleep habits night routines are just as important as morning routines if not more so because they set you up for the following morning habits are composed of cues triggers reminders to do a thing routines the thing and rewards reasons to do the thing example cue alarm at routine go to bed and read journal for an hour reward the activity itself you should enjoy reading journaling your routine should help you wind down i advise no screens because blue light inhibits you from producing the sleep hormone melatonin obviously it doesnt have to be reading you can adjust this to your liking and make sure it works for you morning example cue alarm at from your phone on the other side of the room routine get up turn the alarm off leave your bedroom reward have a cup of coffee to help wake up again adjust this to your liking and make sure it works for you,1,0.757576
170062,eipbsj5,comment,i have been struggling with this for years and i think i have hacked it the night before have a set time to go to bed all days a week make a simple evening routine brush teeth wash face set out clothes get off your phone read journal stretch anything but a screen the morning of get a sun light alarm set for minutes before your goal time it will gently wake you up before beeping set a second alarm across the room i use my phone for your goal time have a glass of water next to your alarm and drink it as soon as you get up leave your room i cant stress this enough go to the kitchen for breakfast or coffee the bathroom for a shower etc,1,0.723073
95344,18a0nja,submission,if youre struggling to get out of bed in the morning try this get a cheap alarm clock it doesnt have to be fancy just functional get a lockable box anything with a key that the alarm clock fits in set the alarm clock to the time you want to get up lock the alarm clock in the box and put the key in the furthest place possible from where you sleep a room downstairs outside etc set an alarm on your phone for minutes before the alarm clock is due to go off and leave your phone by your bed now when the phone alarm wakes you up you have minutes to get up and get the key before the alarm clock in the box goes off by this point youre probably awake enough to carry on with your day enjoy,1,0.722656
446225,jq88qcv,comment,something that has been working for me a little is to start small i believe doing exercise is more about making it a habit that you dont think about rather than something you have to motivate yourself to do every day ive been doing only push ups every day and thats it push ups its not much and if you are expecting to get ripped youll be waiting a long time but the key is to make the idea of exercising as normal as possible in your routine the goal here is that you get yourself used to do those push ups every day so that it starts fe

## Topic 2
Topic 2: job, school, high, skill, college, career, business, community, dream, study, degree, position, company, passion, art

Docs with highest probability: GOOD

Docs with lowest probability: GOOD

In [99]:
topic2 = data[(data.dominant_topic == 2)].sort_values("prob_topic", ascending=False)
print(topic2["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic2[["id","type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       7671
submission    3348
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
249163,gq6nusw,comment,youre fine its a good thing youve had lots of different experiences i wouldnt worry about not having a dream job just look at what kind of entry level jobs seem appealing enough to pursue based on how you want your lifestyle to be im and have only worked in restaurants as a host or server for the most part i dropped out of uni college times when i was younger because i couldnt find a program and career track i wanted to stick to i took some time to work and think about what i should pursue and completed a one year computer foundations certificate college program i plan to upgrade that to a computer programming diploma this fall and am currently working part time on a certified in website creation and design certificate as well i want to enter the software and web development field because of the flexibility and opportunity it brings its a non physical hard skill thats in demand pays well requires little formal education and can be done remotely if needed theres all different kinds of job positions within the field with different focuses front end back end full stack web mobile system architecture project management etc its a skill that can allow someone to own their own businesses work as contractor work salary positions whatever they choose software and it was never a dream career path for me but when i realistically looked at the real jobs in the economy and what field seemed like it had good conditions i had a good aptitude for and what seemed somewhat appealing it stood out youve just got to find a real job and career path with conditions that appeal to you that is in demand or at least attainable but first you need to know whats important to you in a career tell me what do you want in a career,2,0.710269
395921,j3pzycw,comment,you do have to go to college at least in my area all the jobs i looked at while having a bachelors degree actually require more education to even get an entry level job to get experience to move on to next steps i have a bachelors and could only get a job that only requires a ged because a bachelors isnt enough im being paid the minimum of the pay range for this job everyone i know is in this same situation getting stuck in a job with no little promotion opportunity because a bachelors is no longer enough several people at my current job have been there for years and passed on promotions others are exactly like me one guy who didnt go to college has years of work experience and is literally in the exact same job with the same pay as me but i have the option to get a certification within months get a job in the field im educated for and make whats listed as mid range in my current job as a starting rate going back for years could get me double my current pay at entry level which i can never reach in my current job regardless of how long im there because its beyond the max pay of the range for this position,2,0.708793
400892,j5q52v0,comment,one thing to think about is a degree does not mean that you will be working in that field with education degree you can always fall back on it in hard times it allows access to social work hr training departments teaching of course and any other profession that simply requires a degree to start out i recommend that you first get some stability for your self you need to be able to provide for yourself and your children then while you are in college you can use your assignments to research things for your future business ideas i see you have several years of sales so it seems you have the skills to set up your bussiness maybe research online sales methods and to spin it in a way to fit your education how it affected schools or child development trends in youth for sales youll need to be creative and to develop you have to know how to expand you got this graduate get a job and focus on saving then venture out in your business study sales trends in the market your going for ect,2,0.702405
140761,dvl

## Topic 3
Topic 3: friend, relationship, new, alone, hobby, partner, interest, group, fun, conversation, talk, nice, family, together, close

Docs with highest probability: GOOD

Docs with lowest probability: Good enough!

In [100]:
show_example_docs(data,3,False)

type
comment       17381
submission     7351
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
6628,64hls6,submission,hey everyone this is a problem i have been struggling for a couple of years a few years back i was in a group of really good and close friends who were cool to hang out with and i could trust them in my life then i got into some disputes with them and i left the group as a result i have no friends i could really hang out with and trust in needy of friends i decided to start meeting different people and in desperate need of the same relationship i once had i was needy and wanted to rush the relationships so i could have a similar group of close friends now its been about two years and i still dont have a group of close friends or any close friends to be honest i do know a lot of people and have a lot of acquaintances but i dont have a group of friends that i could trust please give me some guidance on what i need to do to find another group of close friends that i can hang out with and trust thank you note i dont see my old group of close friends anymore although i have settled the dispute with some of my old close friends some of the other friends still hate me nonetheless i am not very close to my old group of close friends anymore,3,0.833118
85475,15uio6f,submission,hello everyone i feel into drama with my group of friends that i know for around years we know each other for around years we ride motorbikes together we went skiing together we went on trips together and things like that so we know each other for long time and have done many things together a few months ago we planned a short trip with my then new gf and couples from the same group of friends so i can introduce my gf to two other couples from the group the problem was that i had to cancel my trip because i feel ill and i couldnt go to the trip and because i canceled the trip my friend from the group friend a and his gf also canceled the trip this decision didnt go well with the friend b and his gf from the group and because of that friend b and his gf turned all other friends from the group against my friend a last weekend we all went the hole group for day trip and everyone ignored friend a and not wanted to talk to him except me friend a talked to friend b about this situation and b didnt want to discus or solve this issue so im noticing now that they started ignoring me to because i talked and hanged around my friend a and i really dont know what to do i couldnt ignore my friend a because he is still my friend and im not into this stupid drama i just want to hang around with my friends and ride motorbikes not acting like some high school teenagers we are all around years old but i also dont want to lose my other friends because its hard for me to find new friends did anything like that happened to you what would you do in my position i hope that my post is logical enough my english is not the best,3,0.738898
30755,hn2vex,submission,f i feel really lonely and that depresses me which makes me get into a funk and feel negative i do have friends but i feel i am either not important to them they are too boring or too busy i know this guy i really like talking too but he has sooooo many friends so we dont really hang out we text every day i have another friend who only really cares about his girlfriend two friends who are really busy two friends who live far away so i dont really see any of my friends except for maybe once a month and talk on the phone every now and then it kind of sucks i used to have close friends i would see often but they were really toxic and it was codependent friendships i am happier now alone than having toxic friends that bring me down but sometimes i get really lonely i get depressed i know the easy answer is to just make friends but i think to build solid healthy friendships takes time i also dont want to fall into another toxic friendship so i am cautious about who i let into my life in the meantime how do i not let the loneliness get to me i live with my family so i

## Topic 4
Topic 4: body, healthy, health, mental, weight, porn, food, physical, mental_health, addiction, water, diet, drug, muscle, fat

Docs with highest probability: GOOD

Docs with lowest probability: Not too bad

In [101]:
show_example_docs(data,4,False)

type
comment       8087
submission    1426
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
290529,hgtu5bc,comment,eating sugar makes us want to eat even more sugar you shouldnt avoid sugar at all though however avoid added sugars but instead eat natural sugars that can be found in fruits natural sugar is very healthy and you should include natural sugars in your diet this may help the crave for added sugar go down another tip is to track your nutrition not just calories tracking your added sugar intake will help you lower the amount of added sugar you eat sometimes we dont know there is loads of added sugar in a certain food and we eat it and then start craving added sugar added sugar once eaten makes you want to eat more another thing is avoid diet drinks such as diet coke studies show that drinks like diet coke can make your body think it is consuming sugar when its not which makes you crave added sugar also exercise regularly days a week if possible try to do some cardio along with lifting since not lifting while going on a calorie deficit may make you lose muscle when you burn calories you are usually burning carbs and sugars are carbs so exercising is also important not just for getting your added sugar intake down another extremely important factor of losing that addiction is eating a good amount of protein and fat avoid saturated and trans fats though protein and fat help you get more full than you would with sugar making eating a healthy amount of protein and fat greatly helpful also make sure to get plenty of fibers in your carbs since fibers also help make you full lastly if you end up eating many added sugars in one day dont beat yourself up for it it might not be easy but definitely doable dont add on to your unhealthiness by destroying your mental health if you fail you should almost expect you to fail,4,0.750346
71021,10jbixa,submission,proper eating is essential for maintaining good health and well being here are a few tips to help you eat properly eat a balanced diet eating a balanced diet is essential for maintaining good health a balanced diet should include a variety of fruits and vegetables whole grains lean proteins and healthy fats eating a variety of foods will help ensure that you get all the necessary nutrients your body needs control portion sizes eating too much can lead to weight gain and other health problems be mindful of portion sizes and use smaller plates and bowls to help control the amount of food you eat avoid processed foods processed foods are often high in sugar salt and unhealthy fats they are also typically low in nutrients try to limit your intake of processed foods and opt for whole unprocessed foods instead limit sugar and saturated fat consuming too much sugar and saturated fat can increase the risk of several health problems such as diabetes heart disease and certain cancers try to limit your intake of sugar and saturated fat and opt for healthier options instead stay hydrated drinking enough water is essential for maintaining good health aim for at least glasses of water per day and more if you are physically active plan your meals planning your meals in advance can help you make healthier choices and avoid last minute unhealthy options cook at home cooking at home allows you to control the ingredients and portion sizes of your meals it also saves money be mindful while eating eating mindfully means paying attention to your food savoring each bite and being aware of your bodys signals of hunger and fullness dont skip meals skipping meals can lead to overeating later in the day try to eat regular balanced meal remember eating clean is one of the most important part of staying healthy for your fitness journey your overall health and mental health,4,0.737075
199116,fcux35c,comment,im not a fan of restrictive diets i tried if and failed lol i try to do things that i can keep up with for years instead ive been researching this topic and here are some of my tips grocery shop on a full stomach so youre not tempted to buy junk foo

## Topic 5
Topic 5: social, game, medium, video, social_medium, class, youtube, music, sport, movie, club, video_game, apps, high_school, phone

Docs with highest probability: GOOD

Docs with lowest probability: Good enough

In [102]:
show_example_docs(data,5,True)

type
comment       4262
submission    1278
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
87460,16bwe2d,submission,so as the title says ive been overweight my whole life my dad took me to a nutritionist when i was only or and since then ive been on all kinds of diets and my looks occupy my mind of the time in i have been at my highest weight kg and then i found r loseit community here on reddit and lost since then i weigh about maintaining it and i try to lose more and get more fit because i have no muscle at all the most problematic thing is that i have no life at all i wake up snoozing the alarm and i almost cry that i have to go to work go to work come home lay in bed scroll tiktok instagram reddit and thats whats comfortable for me and its hard for me to leave that comfort zone i am really anxious and a lot of times really negative i want to become better at my job as a developer but deep down i also dont want to i played tennis when i was young took dancing classes sang in choir which i really liked but a lot of my family members dad and sisters made jokes when i did any of that stuff so my confidence now is below zero and i dont want to even try again i feel like im a very boring person with zero interests in life i had social media since facebook and i feel like thats also one of the reasons i lost my interests and attention i used to play wow a lot before i made facebook account i enjoy virtual world and its hard for me to face reality when i dont have phone or laptop with myself i feel im getting a panic attack i tried psychologist but its way too expensive for me now,5,0.164720
10102,86myux,submission,so ive been trying to improve myself any way i could since december when i found the nofap subreddit and after going through some shit not gonna get into it because its not so important anyway so far i achieved the following things i started hanging out more before i wasnt hanging out at all i started reading self improvement books so far i read no more mr nice guy and now i started reading daniel golemans emotional inteligence i started exercising before i wasnt exercising at all at home not at a gym i started taking cold showers every morning before i wasnt washing everyday i made a morning routine wake up stretch my body take a cold shower have breakfast drink coffee i started looking for a job never had a job stopped watching porn masturbating less drinking more water started taking photos for my instagram account and playing guitar more dressing way better i got a better self image i wake up earlier usually around am and now heres what i want to improve next also asking for your help here advice links videos anything that might help me posture getting better at social interactions stop procrastinating stop being lazy read more study better promote my music i make instrumental electronic music but i also play guitar be happier with myself and by myself being less awkward with girls im okay i just have a hard time flirting showing attraction etc this might sound funny but lose my virginity speak clearer louder and without stuttering find a job i would like keep my room and apartment cleaner find a carreer in music um thats about it i know its a lot but i feel like i made a lot of progress so far and i want to get as good as i can i would say my confidence is ok because i think im better than some people my confidence used to be really low but not anymore so any help would be greatly appreciated thanks,5,0.170590
460871,jw4jru4,comment,find some hobbies that you can look foward to and can visually see yourself improving for example a few years ago i started creative and copy writing my writing skills both digitally and physically have improved immensly right now im getting into video editing and posting them online as i want to create a successfull youtube channel just sit back and think for a moment what you really liked to do and then try it could be drawing coding martial arts e commerce online marketing or reading a book on a subject youre interested in n

## Topic 6
Topic 6: woman, girl, man, men, guy, sex, confidence, attractive, date, ugly, confident, personality, face, dude, sexual

Docs with highest probability: GOOD

Docs with lowest probability: not too bad

In [103]:
show_example_docs(data,6,False)

type
comment       7960
submission    1290
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
331606,i81xkxy,comment,straight from his porn her pain helpful things you can learn from porn men can touch their penises during sex women can touch their vulvas during sex spit works for lube some women sometimes desire sex without romance telling each other stories can make sex hotter men can climax using their own hand some people like to look at each other during sex some women think about sex in advance women sometimes use their hand to insert the penis into their vagina men sometimes use their hand to insert their penis into a vagina if the penis comes out during intercourse you can simply put it back in some women like fellatio some women like cunnilingus some men like fellatio some men like cunnilingus some people enjoy having the outside of their anus licked some people enjoy licking their partners anus men and women can enjoy intercourse with a condom people can kiss during sex different people kiss really differently even women who enjoy intercourse may like their clitoris stimulated people can change position during sex vulvas can look really different from each other theres more to a vulva than just a vagina some women use and enjoy vibrators and dildos some men like their balls stimulated during sex watching someone undress can be sexy pregnant women can be sexual whether during intercourse oral or manual sex the clitoris can be important the volume of ejaculate is not related to penis size sex is more than penisvagina intercourse some women have orgasms some people are on very friendly terms with ejaculate some older women and older men are sexual older women can be attractive to younger men and vice versa people can have sex with people of different races heterosexual people can enjoy watching same gender sex women with small breasts can enjoy having their breasts stimulated some people enjoy hair pulling spanking and teasing as part of sex people can smile and talk to each other during sex people can indicate to each other what they like during sex some women shave wax their vulvas others dont some men shave wax their pubic area others dont a man can happily ejaculate outside a vagina people dont have to climax at the same time for sex to be great whatever your sexual fantasy youre not the only one who has it,6,0.681469
129012,dj23uim,comment,i am a female i had to deal with being awkward and tall believing my life would be so much easier if i could blend in with my other girlfriends wearing clothes and choosing my style was hard because i felt everyone could see me later once i settled down and puberty happened boys would only try and date me because tall is cool i became insecure and doubted men didnt know if they liked me or just wanted a trophy to carry around now i own it whatever i am i am some guy calls me amazing because im tall lol ok thanks im told im too tall to be a cute girlfriend lol there are many people who would appreciate my height so move along basing your life on other peoples opinions will get you no where even the juiciest peach in the world will be hated by people that dont like peaches so chillax spend your time thinking about how you can maximize yourself at your height if that involves always wearing boots or tall sneakers or finding clothes and accessories that match your body type and length go for it feel good,6,0.678961
378733,iy8a3yx,comment,woman can sense the grasp a man has in other words a woman can tell if a man can hold on to something such as her its all in the way a man dresses talks carrys himself etc i know you dont like hearing it but its all about confidence and im not talking about just telling yourself you feel confident or believing you are confident its about waking up every dang morning knowing the world is yours a woman will sense that woman tend to steer away from the guy that feels depressed and doesnt believe in himself i do believe in but i also believe a man has to do more because however confident the man is 

## Topic 7
Topic 7: money, parent, kid, home, family, child, house, mom, car, adult, country, wife, brother, dad, mother

Docs with highest probability: GOOD

Docs with lowest probability: good enough

In [104]:
show_example_docs(data,7,False)

type
comment       3129
submission    1116
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
380386,iyrywm7,comment,theres a parent in us and a child in us the child is us but if we step into the role of the parent the child is outside of us say youre at a freezer section at the grocers and you are only picking up frozen veggies the child sees eggos and really really wants eggos you as the child want eggos you as the parent want to save your money and spend it on better things so you can eventually have all the egos the child wants so you explain to the child if youre very good for me and stay out of my way while im working for you you can have a box of eggos next weekend i tried to explain this to my therapist im sick of having to be the child that sees things and wants them and being the parent that tells myself no not right now but there is more freedom as the parent than there is as the child because the child has to live to the will of the parent if the child runs the parent the you as the parent run out of money or get too distracted there was a tedx talk about this but the guy described it as the procrastinator monkey and its the same thing the parts of you that wanna do what a child would wanna do in a situation versus what you know you should do be the parent raise a proper you or the child will raise you it only knows how to raise adult children the child cannot raise an adult,7,0.658802
211716,fp1ua8g,comment,you just simply buy a cheap car with your saving when i was i was on my car dont know if cars are expensive in australia but in uk you can easily buy a cheap car with two weeks wages on minimal wage so buying a car is not that wild comparing to prices of car insurances here on my first car id pay three times more just to stay insured than i paid for my car,7,0.628798
325913,i4axvkr,comment,i dont know which country are you from but thats a process start with a bank that allow you a credit card even if that means moving from one bank to another or look for a fintech that maybe offer that the car insurance the car rent eveything builds score so thats a work on progress use klarna and similar or try to get in monthly payments for things that you need youre not going to build score over night most banks if you present your income employer contract they can sue a card for your income which means that you have a credit card with lets say that thats your monthly salary and when you get the payment basically pays the credit card expenses others offer of the total earnings like credit cards with probably youll need to be with your current employers for like month before asking for a whole credit card scheme but the small ones you can get them,7,0.618885
325673,i42bgn0,comment,save money dont spend money frivolously buy things that you absolutely need if theres something you want that isnt a need make sure you have the money without draining your savings have savings accounts one for rainy day and one towards a big purchase like a car or a house dont buy a brand new car buy used work on your credit get a secured credit card with good rates use it for gas and make on time payments keep your utilization super low avoid credit cards with annual fees high rates etc bank with a credit union in my experience ive gotten much better rates on vehicle loans and credit card than most other banks,7,0.612888
142767,dxc5971,comment,if you really dont want to leave your mother behind maybe find a house with an in law suite one that is detatched from the house or separated like a basement above garage suite or land with an rv on it that way your mother is on property but never in your house my aunt has pretty much done that with most of her kids and their families they have their own apartment withen her house pay rent but live separated,7,0.609640
347555,iim52c4,comment,do get credit cards do not accumulate the debt you will want credit cards on your credit report without a monthly balance owed in order to increase your credit score for buying a house car getting a loan etc the b

## Topic 8
Topic 8: book, comfort, cold, language, information, read, zone, reading, comfort_zone, page, search, shower, psychology, google, recommendation

May seem odd but it makes sense, people talk a lot about cold showers in terms of "they read of their benefits etc etc" so the topic seems to be related to like learning or research about wellbeing, something like that. 

Docs with highest probability: Good

Docs with lowest probability: Good! a lot about learning and information

In [105]:
show_example_docs(data,8,True)

type
comment       1089
submission     299
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
24304,ebv1xe,submission,hey everyone so im a male who had a major psychotic break a year ago and was diagnosed with psychosis im still taking medications since then ive quit school and moved to my aunts and uncles in a foreign country my birth country but i grew up somewhere else while waiting for my parents to move to yet another foreign country i feel like i have no personality whatsoever in social situations my cousin takes me out with her friends i feel like im always on the defensive i never have anything to say and i just nod and smile i dont have the motivation or willpower to do or start anything i just eat smoke and walk around the house checking my phone sometimes every day i try to make a new plan or schedule but if i even start it lasts a couple of days at most my attention span is ruined i used to read and listen to podcasts and watch videos for hours and now if i even pick up a book i can last for only a few pages i am very immature and childish have very low self confidence and despite the books ive read i have no knowledge or opinions on anything at all i have an aspiration to do art in my spare time maybe even live off of being a writer but i have no discernable talent or creativity i just feel so hopeless and lost what should i do where should i start how do i start,8,0.147995
463419,jx05uak,comment,is there anything other than tabletop rpgs that you enjoy or want to learn more about photography dancing cooking classes workshops are great place to meet people socialize in my experience finding something that happens on a schedule and kind of forces people to talk mingle helps study groups book clubs help too if youre in school or something finding people is hard for sure and trying to get out of comfort zone is also anxious among other things but youve already done it once i think you can do it again its possible there might be other tabletop game groups that meet in your city too you can even try that,8,0.182914
253373,gtdku44,comment,reading is great but i think a lot of people especially in this sub and similar communities seem to think you have to read a self help business classic educational book you dont have to read whatever makes you want to read you can read fiction romance fantasy childrens lit magazines whatever i read trashy romance books and ik they arent fancy works of literature but its still reading i could spend the whole day reading a book and reading is what caused me to quit social media tv too and if you get into reading you might pick up writing as a hobby also working out is fun too so long as you do what you like not what is deemed most effective ex i dont like cardio so i dont force myself to do it i lift instead some people bike walk do yoga sports etc also you could consider getting into a specific genre of music learning a language or community service volunteering,8,0.186513
81014,14ohx9p,submission,im not asking for books that are just good or great im asking for books that you found profoundly good i have rich dad poor dad and i only read a few chapters but i really want to get my life back on track before i can even think about finances i started college in but covid fucked me up real bad online learning and exams helped others cuz they cheated but it completely back fired on me things changed when i first went to college from from morning to evening and suddenly staying at home the rest of the years dealing with parents loses etc i developed a dangerously bad pron addiction i jerked off if im stressed out only recently i decided i have to make some changes last weeks im sleeping consistently for hrs during my semester break and in no fap mode started lifting weights with a little cardio cold showers in the morning but still feeling like shit most of the time im gonna start counselling therapy soon but i dont wanna depend on their advice too much ive been binge watching self help videos on yt andrew huberman jordan peterson etc it

## Topic 9
Topic 9: therapy, anxiety, therapist, depression, stress, professional, doctor, trauma, anger, angry, anxious, adhd, medication, depressed, symptom

Docs with highest probability: GOOD

Docs with lowest probability: Good!

In [106]:
show_example_docs(data,9,True)

type
comment       1332
submission     217
Name: count, dtype: int64


,id,type,cleaned_text,dominant_topic,prob_topic
118980,d6bw0v0,comment,first of all you have to accept that you can live alone you dont need anyone just stop worrying about havent got any friends or girlfriends if you try to change something in your life already you become more social and propably you will have got a lot of friends just be patient please using roaccutane could help your acne problem this is the modt effective treatment in acne treatment but see your doctor for using this durg because while you are using roaccutane you should have blood test in hospital during processes you should be in contact with your doctor you can do sport you dont need go to the gym if you dont have enough money you can use any park or even your home i using mark lauren body weight training app downloaded in appstore or you can get another application its up to you most important thing is never give up you must do your sport regularly what did i say be patient losing weight could take a time you can watch actualized channel in youtube for selfesteem psychological problems and social skills itve helped me finally you are very young still you can find job and your future is bright if you need money just get out your home and search some job which suitible for you at last you can find job dont worry about that sorry for my bad english,9,0.155174
17889,bihq97,submission,hey guys i am trying to start self improving myself but need some guidance on where to start my whole life is a mess i have social anxiety and depression becuase of my mental health issues i dropped out of college i am a year old dude still living with my parents i want to move out but have no money i recently got a job at a cellphone company as a sales rep i got this job to improve my social skills and fight my social anxiety it is going to be tough but i have to do it develop as a person and have to get some income in since i am broke i wake up lost and unmotivated everyday i am not very disciplined i try to sleep early but i always end up using my laptop until or am and falling asleep and waking up late and groggy my diet is horrible my situation at home is not the best me and my dad argue a lot i just do not know where to start because my life and my mental health is all over the place i have no friends and no support system to talk to when i feel sad or down in life i just feel super lost in life i have anxiety attacks few times a week i just need some advice from you guys who are doing well where should i start first and should be my plan thank you,9,0.168230
493431,k93xotp,comment,what is your goal tackling social anxiety if so and if you havent read rejection proof its a good read and talks about ideas like this and the authors experiences i ended up finding a local group to practice rejection proofing with and it was beneficial for my social anxiety,9,0.177848
82453,152i3uf,submission,hey guys so im years old and i want to really improve on my life in every aspect especially into improving my social life dating life and professional life i just dont want to deal with regrets anymore i dealt with regrets of missing out and doing nothing instead of staying home for many years ive struggled with mental health for so many years so ive always stayed home i did try college but withdrew due to my ocd social anxiety and depression im going back to college in september majoring in spanish english translation i started working at and left my job after almost two years and found another job but only last weeks because of my ocd spike but i got the help i needed and i do group therapy once a month i also see a regular therapist and im doing my best to get better this summer i slowly began to do more things i began to run again and been fairly consistent this time going out with the few friends i have and doing some activities so far my summer has not been bad but i really want to make the most of my and i want to go out to socialize more make more friends eliminate my social anxi